In [3]:
import numpy as np 
import pandas as pd 
import os
import tensorflow as tf
import tensorflow.keras
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from PIL import Image
import warnings
warnings.filterwarnings("ignore")


image_size = [224,224]
data_path = 'Data'
conv = VGG16(input_shape= image_size+[3],weights='imagenet',include_top=False)
conv.output


x = conv.output
x = GlobalAveragePooling2D()(x)


x = Dense(1024,activation='relu')(x)
x = Dense(1024,activation='relu')(x)
x = Dense(512, activation='relu')(x)


pred = Dense(2,activation='softmax')(x)
model = Model(inputs = conv.input,outputs=pred)

model.summary()


for layer in conv.layers:
    layer.trainable = False

train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.24)

train_generator=train_datagen.flow_from_directory('D:/Jupyter/Projects/Brain Tumor Detector/train', target_size=(224,224), color_mode='rgb', shuffle=True, subset='training', batch_size=64, class_mode='categorical')
val_generator = train_datagen.flow_from_directory('D:/Jupyter/Projects/Brain Tumor Detector/val', target_size=(224,224), color_mode='rgb', shuffle=True, subset='validation', batch_size=64, class_mode='categorical')

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])


print(train_generator.n)
print(train_generator.batch_size)
print(242//32)


step_size_train=train_generator.n//train_generator.batch_size
r = model.fit_generator(generator=train_generator, steps_per_epoch=step_size_train, epochs=20, validation_data=val_generator)

#save model
from tensorflow.keras.models import load_model

#tensorflow.keras.models.save_model(model,'mri_tumor.h5', overwrite=True,include_optimizer=True)

model.save('mri_tumor.h5')

'''#load model
model = load_model('/root/glio.ai/models/mri_tumor.h5') 
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator=train_datagen.flow_from_directory('/root/glio.ai/data/braintumordata/brain_tumor_dataset', 
target_size=(224,224), 
color_mode='rgb', 
shuffle=True, 
batch_size=64, 
class_mode='categorical')'''



## plot loss

plt.plot(r.history['loss'], label='training loss')
plt.legend(['Training Loss'])
plt.show()
plt.savefig('LossVal_loss')

## plot accuracy

plt.plot(r.history['accuracy'])
plt.title('Model Accuracy')
plt.legend(['Training Accuracy'])
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()



Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

KeyboardInterrupt: 

In [23]:
## test model with separate test dataset

model = load_model('mri_tumor.h5')

# route to any of the labaled malignant images that model hasn't seen before 
img_path = ('D:/Jupyter/Projects/Brain Tumor Detector/Images to test/no 8.jpg')
img = tensorflow.keras.preprocessing.image.load_img(img_path, target_size=(224,224))
x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)
img_data = preprocess_input(x)

# make prediction
rs = model.predict(img_data)
print(rs)


rs[0][0]

rs[0][1]


if rs[0][0] == 0:
    prediction = 'Warning! : This Image has Brain tumor'
else:
    prediction = 'This image is of Healthy brain.'

print(prediction)

[[0.00438424 0.9956157 ]]
This image is of Healthy brain.
